In [1]:
import pandas as pd
import os

In [2]:
i = 1
df = pd.read_csv(f"output/simulation_results.csv")
df

,experiment_id,sim_id,temp,hum,water_capacity,toilet_capacity,average_time,water_station_counter,energy_station_counter,bathroom_station_counter,medical_station_counter
0,0,0,L,L,50,37,162.858738,1293,108,496,12
1,1,0,L,L,75,37,163.015835,1271,116,519,17
2,2,0,L,H,50,25,164.106998,1289,137,517,15
3,3,0,L,H,75,25,163.990238,1281,132,502,20
4,4,0,H,L,50,25,167.166541,1286,117,515,17
...,...,...,...,...,...,...,...,...,...,...,...
75,3,9,L,H,75,25,163.997730,1278,123,487,13
76,4,9,H,L,50,25,166.737981,1252,90,501,12
77,5,9,H,L,75,25,166.609732,1267,130,505,15
78,6,9,H,H,50,37,168.090162,1277,115,504,13


In [3]:
group_cols = ["experiment_id", "temp", "hum", "water_capacity", "toilet_capacity"]
mean_df = df.groupby(group_cols).mean().reset_index().drop(columns=['sim_id'])
std_dev_df = df.groupby(group_cols).std().reset_index().drop(columns=['sim_id'])

mean_df = mean_df.rename(columns={col: col + '_mean' for col in mean_df.columns if col not in group_cols})
std_dev_df = std_dev_df.rename(columns={col: col + '_std' for col in std_dev_df.columns if col not in group_cols})


In [4]:
result = pd.merge(mean_df, std_dev_df, on=group_cols, how='inner')

result

,experiment_id,temp,hum,water_capacity,toilet_capacity,average_time_mean,water_station_counter_mean,energy_station_counter_mean,bathroom_station_counter_mean,medical_station_counter_mean,average_time_std,water_station_counter_std,energy_station_counter_std,bathroom_station_counter_std,medical_station_counter_std
0,0,L,L,50,37,163.006219,1274.4,115.5,507.1,12.6,0.132717,17.321791,13.201431,22.708050,3.717825
1,1,L,L,75,37,163.026948,1263.3,112.6,499.0,14.4,0.127939,26.123850,14.041209,18.073922,5.081557
2,2,L,H,50,25,164.075576,1276.9,124.6,494.8,13.5,0.104665,17.622271,10.057612,20.922078,3.894440
3,3,L,H,75,25,163.976057,1283.0,121.8,505.6,13.4,0.144033,18.239152,11.923832,19.540272,3.687818
4,4,H,L,50,25,167.005277,1264.9,112.5,502.4,12.9,0.169896,15.835614,10.352133,17.004575,4.228212
5,5,H,L,75,25,166.951423,1279.3,121.0,511.0,14.3,0.168725,18.850582,10.077478,19.635569,3.335000
6,6,H,H,50,37,167.969436,1268.2,118.4,502.9,14.0,0.167544,26.869231,10.772393,19.456219,2.788867
7,7,H,H,75,37,167.973474,1277.8,119.3,508.2,13.3,0.089396,21.332292,12.693393,15.526322,4.110961


In [5]:
stds_list = result.average_time_std.values
means_list = result.average_time_mean.values

for m, s in zip(means_list,stds_list):
    n = 10
    h = (2.262*s)/(n**0.5) #t(alpha=0.05,n=10)
    h_star = 0.05*m
    n_star = n*(h/h_star)**2
    print(m, s, h, h_star, n_star)
    if n_star<n:
        print("We cannot reduce the number of replications")
        n_star = n
    else:
        print("Replication Calculation Successful")
    

163.00621879021259 0.132716724475364 0.09493322947083925 8.15031093951063 0.0013567135262338513
We cannot reduce the number of replications
163.02694841968565 0.12793897647566224 0.09151567189471231 8.151347420984283 0.0012604689089414879
We cannot reduce the number of replications
164.0755759971536 0.10466469559897845 0.07486741105216015 8.20377879985768 0.0008328326578555162
We cannot reduce the number of replications
163.976057411397 0.14403326074280276 0.1030280294181606 8.19880287056985 0.0015791009601705663
We cannot reduce the number of replications
167.00527701124548 0.1698957006156127 0.12152761904280511 8.350263850562275 0.00211811522384116
We cannot reduce the number of replications
166.95142313184186 0.1687251258236734 0.12069029845811435 8.347571156592094 0.0020903762525766624
We cannot reduce the number of replications
167.9694357445117 0.16754373786036889 0.11984524313401845 8.398471787225587 0.0020362967416911516
We cannot reduce the number of replications
167.973474368

In [6]:
# Encode the factors (low = -1, high = 1)
result['temp_encoded'] = result['temp'].apply(lambda x: -1 if x == 'L' else 1)
result['hum_encoded'] = result['hum'].apply(lambda x: -1 if x == 'L' else 1)
result['water_capacity_encoded'] = result['water_capacity'].apply(lambda x: -1 if x == 50 else 1)
result['toilet_capacity_encoded'] = result['toilet_capacity'].apply(lambda x: -1 if x == 25 else 1)


# Create interaction terms
result['temp_hum_interaction'] = result['temp_encoded'] * result['hum_encoded']
result['temp_water_interaction'] = result['temp_encoded'] * result['water_capacity_encoded']
result['temp_toilet_interaction'] = result['temp_encoded'] * result['toilet_capacity_encoded']
result['hum_water_interaction'] = result['hum_encoded'] * result['water_capacity_encoded']
result['hum_toilet_interaction'] = result['hum_encoded'] * result['toilet_capacity_encoded']
result['water_toilet_interaction'] = result['water_capacity_encoded'] * result['toilet_capacity_encoded']


# Calculate the average response for high and low levels
def calculate_effect(df, column):
    high_level_mean = df[df[column] == 1]['average_time_mean'].mean()
    low_level_mean = df[df[column] == -1]['average_time_mean'].mean()
    effect = high_level_mean - low_level_mean
    return effect

# Calculate the interaction effects
def calculate_interaction_effect(df, interaction_column):
    high_level_mean = df[df[interaction_column] == 1]['average_time_mean'].mean()
    low_level_mean = df[df[interaction_column] == -1]['average_time_mean'].mean()
    effect = high_level_mean - low_level_mean
    return effect



In [7]:
# Calculate the main effects
temp_effect = calculate_effect(result, 'temp_encoded')
hum_effect = calculate_effect(result, 'hum_encoded')
water_capacity_effect = calculate_effect(result, 'water_capacity_encoded')
toilet_capacity_effect = calculate_effect(result, 'toilet_capacity_encoded')

# Print the effects
print(f"Effect of temp: {temp_effect:.4f}")
print(f"Effect of hum: {hum_effect:.4f}")
print(f"Effect of water_capacity: {water_capacity_effect:.4f}")
print(f"Effect of toilet_capacity: {toilet_capacity_effect:.4f}")

Effect of temp: 3.9537
Effect of hum: 1.0012
Effect of water_capacity: -0.0322
Effect of toilet_capacity: -0.0081


In [8]:
# Calculate the interaction effects
temp_hum_effect = calculate_interaction_effect(result, 'temp_hum_interaction')
temp_water_effect = calculate_interaction_effect(result, 'temp_water_interaction')
temp_toilet_effect = calculate_interaction_effect(result, 'temp_toilet_interaction')
hum_water_effect = calculate_interaction_effect(result, 'hum_water_interaction')
hum_toilet_effect = calculate_interaction_effect(result, 'hum_toilet_interaction')
water_toilet_effect = calculate_interaction_effect(result, 'water_toilet_interaction')

# Print the interaction effects
print(f"Interaction effect of temp and hum: {temp_hum_effect:.4f}")
print(f"Interaction effect of temp and water capacity: {temp_water_effect:.4f}")
print(f"Interaction effect of temp and toilet capacity: {temp_toilet_effect:.4f}")
print(f"Interaction effect of hum and water capacity: {hum_water_effect:.4f}")
print(f"Interaction effect of hum and toilet capacity: {hum_toilet_effect:.4f}")
print(f"Interaction effect of water capacity and toilet capacity: {water_toilet_effect:.4f}")

Interaction effect of temp and hum: -0.0081
Interaction effect of temp and water capacity: 0.0072
Interaction effect of temp and toilet capacity: 1.0012
Interaction effect of hum and water capacity: -0.0156
Interaction effect of hum and toilet capacity: 3.9537
Interaction effect of water capacity and toilet capacity: 0.0445
